Name: Yara Hossam El-Din

Sec: 2

BN: 48

# **Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
from numpy import log2
import pprint
epslon = np.finfo(float).eps

# **Tree Implementation**

In [2]:
def get_entropy(df):
  """gets the entropy of a dataset"""
  entropy=0
  target=df.columns[-1]
  total=len(df[target])
  unique_values=df[target].unique()
  for val in unique_values:
   count=len(df[df[target]==val])
   p = count/total
   entropy += p * log2(p)
  return -entropy

In [3]:
def get_feature_entropy(df,feature):
  """returns the entropy of dataset grouped by values of a certain feature"""
  feature_entropy=0
  target=df.columns[-1]
  total=len(df[target])
  target_unique_values=df[target].unique()
  feature_unique_values=df[feature].unique()
  for feature_val in feature_unique_values:
    feature_val_count=len(df[df[feature]==feature_val])
    weight=feature_val_count/total
    val_entropy=0
    for target_val in target_unique_values:
      targ_given_feature_count=len(df[(df[feature]==feature_val )& (df[target]==target_val)])
      #to avoid division by 0 or log(0), we use epslon, a very small +ve float to add it to our deniminator
      p=targ_given_feature_count/feature_val_count+epslon
      val_entropy+=p*log2(p+epslon)
    feature_entropy+=(weight)*-val_entropy  
  return feature_entropy

In [4]:
def best_IG(df):
  """returns the feature with the highest information gain in a dataframe"""
  IG_dict={}
  dataset_entropy=get_entropy(df)
  for feature in df.columns[:-1]:
    feature_entropy = get_feature_entropy(df,feature)
    feature_IG = dataset_entropy-feature_entropy
    IG_dict[feature]=feature_IG
    max_IG = max(IG_dict, key=IG_dict.get)
  return max_IG

In [5]:
def decision_tree(df):
  """returns a decision tree as a dictionary with the best tree nodes in order"""
  tree={}
  target=df.columns[-1]
  subtree = best_IG(df)
  if len(tree)==0 :
    tree[subtree]={}
    feature_unique_values=df[subtree].unique()
    for val in feature_unique_values:
      subset=df[df[subtree]==val].drop(subtree,axis=1)
      entropy=get_entropy(subset)
      target_unique_values=subset[target].unique()
      count=len(target_unique_values)
      if count==1:
        tree[subtree][val] = target_unique_values[0]
      else:
        tree[subtree][val] = decision_tree(subset)
  return tree  

# **Testing** 




### **1. Weather dataset**

In [6]:
#Now we test our implementation on two datasets
df_weather=pd.read_csv("weather_data.csv")
df_weather

,Outlook,Temperature,Humidity,Wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [7]:
tree_weather = decision_tree(df_weather)
#to print the dictionary in a neat way, we use pprint
pprint.pprint(tree_weather)

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


#### We can see that the results match manual calculations.
#### For manual calculations on the weather dataset, refer to this video: https://www.youtube.com/watch?v=coOTEc-0OGw

### **2. Grades dataset**

In [8]:
df_grades=pd.read_csv("grades_data.csv")
df_grades

,Early registration,Finished homework,Senior,Likess Coffee,Liked the last homework,A
0,1,1,0,0,1,1
1,1,1,1,0,1,1
2,0,0,1,0,0,0
3,0,1,1,0,1,0
4,0,1,1,0,0,1
5,0,0,1,1,1,1
6,1,0,0,0,1,0
7,0,1,0,1,1,1
8,0,0,1,0,1,1
9,1,0,0,0,0,0


In [9]:
tree_grades = decision_tree(df_grades)
pprint.pprint(tree_grades)

{'Finished homework': {0: {'Likess Coffee': {0: {'Senior': {0: 0,
                                                            1: {'Liked the last homework': {0: 0,
                                                                                            1: 1}}}},
                                             1: 1}},
                       1: {'Early registration': {0: {'Senior': {0: 1,
                                                                 1: {'Liked the last homework': {0: 1,
                                                                                                 1: 0}}}},
                                                  1: 1}}}}


####Again, we see that the tree matches tha manually calculated one (manual tree on grades dataset is attached in pdf)